In [1]:
import os, sys
os.chdir("..")
sys.path.append("..")

from utils.support import pkl_load
from train.run import MyDataset
from train.versions.augmentation import get_augmentation
from train.versions.model import get_model

import os
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [20]:
# valid metrics per hypohesis
for h in list(range(1, 8)): # + ["debug"]:
    p = pkl_load(f"../output/models/hypothesis-{h}/fold-1/progress.pkl")
    print(f"{h} - {p.valid_accuracy.max()} | {p.valid_loss.min()}", "")

1 - 0.785 | 0.4937687518610619 
2 - 0.96375 | 0.1022379537772488 
3 - 0.9725 | 0.0907153371499115 
4 - 0.97 | 0.08921746774294795 
5 - 0.97 | 0.08568617809048248 
6 - 0.965 | 0.09340458868674367 
7 - 0.96125 | 0.1071347354391017 


In [17]:
# valid metrics per fold
h = 5
for f in list(range(1, 6)): # + ["debug"]:
    p = pkl_load(f"../output/models/hypothesis-{h}/fold-{f}/progress.pkl")
    print(f"{f} - {p.valid_accuracy.max()} | {p.valid_loss.min()}")

1 - 0.97 | 0.08568617809048248
2 - 0.96875 | 0.09439045888191686
3 - 0.96375 | 0.10262218771850712
4 - 0.96 | 0.10195685172744788
5 - 0.97875 | 0.06239464360918646


In [19]:
# train metrics per fold
h = 5
for f in list(range(1, 6)): # + ["debug"]:
    p = pkl_load(f"../output/models/hypothesis-{h}/fold-{f}/progress.pkl")
    print(f"{f} - {p.train_accuracy.max()} | {p.train_loss.min()}")

1 - 0.984375 | 0.05525702387623554
2 - 0.985625 | 0.04988225664510223
3 - 0.9746875 | 0.07536254067994917
4 - 0.9734375 | 0.07856304098562873
5 - 0.9765625 | 0.07142878001491916


In [ ]:
p4[p4["valid_loss"] == p4["valid_loss"].min()]

In [ ]:
p4[p4["train_loss"] == p4["train_loss"].min()]

In [ ]:
p4

In [ ]:
p4["train_loss"].min()

In [ ]:
p4["train_accuracy"].max()

In [ ]:
p4["valid_accuracy"].max()

In [ ]:
cfg = {
    "model_version": 1,
    "model_weights": "../output/models/v-1/fold-1/model.pt",
    
    "augmentation_version": 1,
    
    "device": "cuda",
    "batch_size": 4000,
    "n_jobs": 4,
}

In [ ]:
dct = pkl_load(Path("../output/crossval_split/crossval_dct.pkl"))

df = dct[1]["valid"]
dataset = MyDataset(df, get_augmentation(cfg["augmentation_version"])["valid"])
dataloader = DataLoader(dataset, batch_size=cfg["batch_size"], shuffle=False, num_workers=cfg["n_jobs"])

In [ ]:
model = get_model(cfg["model_version"], cfg["model_weights"])
model.to(cfg["device"])
pass

In [ ]:
with torch.no_grad():
    model.eval()
    
    pred = np.array([])
    for x, y in dataloader:
        x, y = x.to(cfg["device"]), y.to(cfg["device"])
        p = model.forward(x)
        p = torch.sigmoid(p)
        p = p.to("cpu").numpy()
        pred = np.append(pred, p)
df["pred_proba"] = pred
df["pred"] = (df["pred_proba"] > 0.5).astype(int)

In [ ]:
def preview(df, ncols, size):
    nrows = int(len(df) / ncols) + (len(df) % ncols > 0)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                            figsize=(ncols*size, nrows*size))
    fig.patch.set_facecolor('white')
    for ax in axs.flatten():
        ax.axis('off')
    for ax, (_, row) in zip(axs.flatten(), df.iterrows()):
        img = cv2.imread(str(row["cew_img"]), cv2.IMREAD_GRAYSCALE)
        ax.imshow(img, cmap=plt.get_cmap('gray'))
        ax.set_title(f"True-{row.label} | Pred-{round(row.pred_proba, 3)}")

    plt.tight_layout()
    plt.show()

In [ ]:
preview_df = df[(df["label"] != df["pred"]) & (df["label"] == 1)]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[(df["label"] != df["pred"]) & (df["label"] == 0)]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[
    (df["pred"] == 1) &
    
    (df["pred"] == df["label"]) &
    (df["pred_proba"] > 0.3) &
    (df["pred_proba"] < 0.7)
]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)

In [ ]:
preview_df = df[
    (df["pred"] == 0) &
    
    (df["pred"] == df["label"]) &
    (df["pred_proba"] > 0.3) &
    (df["pred_proba"] < 0.7)
]
print(f"Total images: {preview_df.shape[0]}")
preview(preview_df, 5, 2)